<a href="https://colab.research.google.com/github/safari-mohammadreza/MWL_DeepLearning/blob/main/CNN%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this code, I reserved data of 5 subjects out of 48 total subjects for testing.

# Initializer

In [ ]:
# 1. mount drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# 2. set directory of data
%cd /content/gdrive/My Drive/your_path/dDTF_images/
base_dir='/content/gdrive/My Drive/your_path/dDTF_images/3classes_sub'

In [ ]:
# 3. import libraries

import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer,Dense, Dropout, Input, Activation, TimeDistributed, Reshape
from tensorflow.keras.layers import  GRU, Bidirectional
from tensorflow.keras.layers import  Conv1D, Conv2D, MaxPooling2D, Flatten, BatchNormalization, LSTM, ZeroPadding2D, GlobalAveragePooling2D, SpatialDropout2D, GlobalMaxPool1D,Convolution1D
from tensorflow.keras.callbacks import History
from tensorflow.keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils

import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix
import time
import math
import random
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from natsort import natsorted

import cv2
import pickle
from PIL import Image

In [ ]:
# 4. define some methods

def smooth_curve(points, factor=0.0):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

def plot_results(history):
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(acc) + 1)
  %matplotlib inline
  plt.plot(epochs, smooth_curve(acc), 'bo', label='Training acc')
  plt.plot(epochs, smooth_curve(val_acc), 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  plt.plot(epochs, smooth_curve(loss), 'bo', label='Training loss')
  plt.plot(epochs, smooth_curve(val_loss), 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()

# def calculate_test_accuracy(X_test, Y_test, model):
#   fold_results = {}
#   Y_test = np_utils.to_categorical(Y_test)
#   predicted_probas = model.predict(np.array(X_test),steps = len(X_test))
#   fold_results['predicted_probas'] = predicted_probas
#   binary_prediction=[]
#   binary_prediction0=[]
#   for i in range(len(predicted_probas)):
#       binary_prediction0 = 1 if predicted_probas[i,:].argmax()>=0.5 else 0
#       # binary_prediction0 = np.round(predicted_probas)
#       binary_prediction.append(binary_prediction0)
#   fold_results['binary_prediction'] = binary_prediction
#   conf_mat = confusion_matrix(binary_prediction, Y_test)
#   fold_results['confusion_matrix'] = conf_mat
#   acc = np.sum(conf_mat.diagonal()) / np.sum(conf_mat)
#   fold_results['test_accuracy'] = acc
#   print('Overall accuracy: {:.2f} %'.format(acc*100))

#   return fold_results


# def calculate_best_validation_accuracy(history):
#   # Find the index of epoch with the highest validation accuracy
#     best_epoch = history.history['val_acc'].index(max(history.history['val_acc']))
#     # Get the best validation accuracy
#     best_val_acc = history.history['val_acc'][best_epoch]

#     print("Best validation accuracy: {:.2f}%".format(best_val_acc * 100))

#     return best_val_acc


def calculate_test_accuracy_simple(x_test, y_test, model):

  Y_test = np_utils.to_categorical(Y_test)
  accuracy = model.predict(np.array(X_test),steps = len(X_test))

  # _,accuracy = model.evaluate(x_test, to_categorical(y_test.reshape((-1,1))))

  print("Accuracy: {:.2f}%".format(accuracy * 100))

  return accuracy


def create_model():
  conv_base = ResNet50(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
  conv_base.trainable = True

  model = models.Sequential()
  model.add(conv_base)
  model.add(layers.Flatten())
  model.add(layers.Dense(32)) #,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))

  cnn = models.Model(model.input, model.output) # dim of output is [number of input,32] -- 32 comes from last later of model. [5, 32]

  seq_input = Input(shape=(5,224,224,3))
  encoded_sequence = TimeDistributed(cnn)(seq_input) #TimeDistributed, break the video to pictures and feed them to cnn, then concat the results.
  encoded_sequence = Bidirectional(LSTM(32, return_sequences=True))(encoded_sequence) #if return_seq = True, it gives a seq in output. in bidirectional LSTM we have 32 sequence in each side

  #encoded_sequence = Dropout(rate=0.1)(encoded_sequence)
  encoded_sequence = Bidirectional(LSTM(32, return_sequences=False))(encoded_sequence) #if return_seq = False, it gives the last output only
  out = Dense(2, activation="softmax")(encoded_sequence)
  #out = Convolution1D(1, kernel_size=1, activation="sigmoid", padding="same")(encoded_sequence)

  cnn_lstm = models.Model(seq_input, out) # with .Model we give the input and output of the model and it creates the model itself.
  cnn_lstm.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(learning_rate=1e-4),metrics=['acc'])

  return cnn_lstm


# Load data

In [ ]:
# 5'. load images data from train and test folders

hi_s1 = '/content/gdrive/My Drive/your_path/dDTF_images/3classes_sub/theta/hi/s1'
train_hi_data = os.listdir(hi_s1)
train_hi_data = natsorted(train_hi_data)
print(np.array(train_hi_data).shape)

hi_s1 = '/content/gdrive/My Drive/your_path/dDTF_images/3classes_sub/theta/hi/s1'
train_hi_data = os.listdir(train_hi_dir)
train_hi_data = natsorted(train_hi_data)
print(np.array(train_hi_data).shape)


train_lo_dir = '/content/gdrive/My Drive/your_path/dDTF_images/2classes/gama/train/lo'
train_lo_data = os.listdir(train_lo_dir)
train_lo_data = natsorted(train_lo_data)
print(np.array(train_lo_data).shape)

test_hi_dir = '/content/gdrive/My Drive/your_path/dDTF_images/2classes/gama/test/hi'
test_hi_data = os.listdir(test_hi_dir)
test_hi_data = natsorted(test_hi_data)
print(np.array(test_hi_data).shape)

test_lo_dir = '/content/gdrive/My Drive/your_path/dDTF_images/2classes/gama/test/lo'
test_lo_data = os.listdir(test_lo_dir)
test_lo_data = natsorted(test_lo_data)
print(np.array(test_lo_data).shape)

In [ ]:
# 6'. Resize images. this cell may take some time

train_hi_resized = []
for img in train_hi_data:
    with Image.open(train_hi_dir+"/"+img) as im:
      resized_im = im.resize((224,224))
      train_hi_resized.append(resized_im)


train_lo_resized = []
for img in train_lo_data:
    with Image.open(train_lo_dir+"/"+img) as im:
      resized_im = im.resize((224,224))
      train_lo_resized.append(resized_im)


test_hi_resized = []
for img in test_hi_data:
    with Image.open(test_hi_dir+"/"+img) as im:
      resized_im = im.resize((224,224))
      test_hi_resized.append(resized_im)


test_lo_resized = []
for img in test_lo_data:
    with Image.open(test_lo_dir+"/"+img) as im:
      resized_im = im.resize((224,224))
      test_lo_resized.append(resized_im)


In [ ]:
# 7. create images_data that is a collection of all classes

images_data = train_hi_resized + train_lo_resized + test_hi_resized + test_lo_resized # 1591,1591,185,185

In [ ]:
# 8. dump images_data as Images_48subjects.

with open(base_dir+'Images_gama', 'wb') as file_pi1:
    pickle.dump(images_data, file_pi1)

In [ ]:
# 9. load the images_data from drive if you don't wanna run from first cell.

with open(base_dir+'Images_gama', 'rb') as f:
    images_data = pickle.load(f)


print(type(images_data[0]));
print(np.shape(images_data[0]))

In [ ]:
# 10. convert members to np.array

images_data = [np.asarray(img) for img in images_data]

# **Prepare the video Samples**
prepare video samples from previously loaded Images_data

In [ ]:
# 11. If you start running from 9th cell you need run this cell to define X_q1 and X_q2

train_hi_resized = images_data[:1591]
train_lo_resized = images_data[1591:2*1591]
test_hi_resized = images_data[2*1591:(2*1591)+185]
test_lo_resized = images_data[(2*1591)+185:]

print(np.shape(train_hi_resized[0]))
print(np.shape(train_hi_resized))
print(np.shape(test_hi_resized))

In [ ]:
# 12. create video data

train_video_hi = []
subj = 0
stride = 1
init_frame = 0
frame_length = 5 # 5 images per video
for subj in range(43): # number of subjects
  for img in range(37): #stride 1/number of images of each subject / / 43 * (37-5+1) = 1419 total video samples per freq. band (ie. gama)
       if img + frame_length > 37:
          break
       tmp_video = train_hi_resized[subj*37+img:subj*37+img+frame_length]
       # first 0-5 second 1-6 third 2-7 last 32-37
       # 33 movies for each patient. 33*48 movie in total.
       train_video_hi.append(tmp_video)

print(np.shape(train_video_hi)) # The result will be (1419, 5, 224, 224, 3) that is 1419 videos that each of them consist of 5 (224,224,3) images
print(img)


train_video_lo = []
subj = 0
stride = 1
init_frame = 0
frame_length = 5
for subj in range(43):
  for img in range(37):
       if img + frame_length > 37:
          break
       tmp_video = train_lo_resized[subj*37+img:subj*37+img+frame_length]
       train_video_lo.append(tmp_video)

print(np.shape(train_video_lo))
print(img)


test_video_hi = []
subj = 0
stride = 1
init_frame = 0
frame_length = 5
for subj in range(5): # data of 5 subjets are reserved for test
  for img in range(37):
       if img + frame_length > 37:
          break
       tmp_video = test_hi_resized[subj*37+img:subj*37+img+frame_length]
       test_video_hi.append(tmp_video)

print(np.shape(test_video_hi))
print(img)

test_video_lo = []
subj = 0
stride = 1
init_frame = 0
frame_length = 5
for subj in range(5):
  for img in range(37):
       if img + frame_length > 37:
          break
       tmp_video = test_lo_resized[subj*37+img:subj*37+img+frame_length]
       test_video_lo.append(tmp_video)

print(np.shape(test_video_lo))
print(img)

In [ ]:
# 14. create video_data that is a collection of all classes

video_data = train_video_hi + train_video_lo + test_video_hi + test_video_lo
print(np.shape(video_data))


In [ ]:
# clear memory

train_video_hi, train_video_lo, test_video_hi, test_video_lo = [],[],[],[] # we achieved video_data so we clear memory by removing unused variables
images_data, train_hi_resized, train_lo_resized, test_hi_resized, test_lo_resized = [], [], [], [], []

In [ ]:
# 15. dump video_classes

with open(base_dir+'video_gama', 'wb') as file_pi1:
    pickle.dump(video_data, file_pi1)

In [ ]:
# 16. load video_classes from drive

with open(base_dir+'video_gama', 'rb') as f:
    video_data = pickle.load(f)

    print(np.shape(video_data))


In [ ]:
# 18'. set train/test data and labels
X_train = video_data[:1419*2]
X_test = video_data[1419*2:]
Y_train = 1419*[1] + 1419*[0] # hi_label=1 , lo_label=0
Y_test = 165*[1] + 165*[0]

print(np.shape(X_test)) # (165*2, 5, 224, 224, 3)

In [ ]:
# clear memory

Images_data = []
video_data = []

# Training

In [ ]:
# Training and validating model

x_train = np.array(X_train)
x_val = np.array(X_test)
y_train = np.array(Y_train)
y_val = np.array(Y_test)
y_train = np_utils.to_categorical(y_train)
y_val = np_utils.to_categorical(y_val)

model = create_model()

history = model.fit(x_train,y_train,epochs=25, batch_size=10, validation_data = (x_val,y_val), shuffle=True)

plot_results(history)

In [ ]:
# Test and accuracy

x_test = np.array(X_test)
y_test = np.array(Y_test)

calculate_test_accuracy_simple(x_test, y_test, model)